In [ ]:
import copy
import json
import math
import random
import sys
import numpy as np

from PIL import Image
from PIL import ImageDraw
from PIL import ImageFilter

COLOUR_BLACK = (0, 0, 0, 255)
COLOUR_WHITE = (255, 255, 255, 255)
POLY_MIN_POINTS = 3
POLY_MAX_POINTS = 5
POLYGONS = 50
class Shape(object):
    def __init__(self, colour=None, points=[]):
        self.colour = colour
        self.points = points

    def mutate(self, size):
        rand = random.random()
        if rand <= 0.5:
            idx = random.randrange(0, 4)
            value = random.randrange(0, 256)
            colour = list(self.colour)
            colour[idx] = value
            self.colour = tuple(colour)
        else:
            idx = random.randrange(0, len(self.points))
            point = rand_point(size[0], size[1])
            self.points[idx] = point

class Chromosome(object):
    def __init__(self, imageSize, shapes=[]):
        self.imageSize = imageSize
        self.shapes = shapes
        self.generation = 0
    
    def __iter__(self):
        for shape in self.shapes:
            yield shape
    
    def __getitem__(self, idx):
        if idx < len(self.shapes):
            return self.shapes[idx]
        else:
            raise IndexError

    def draw(self, background=COLOUR_BLACK, show=False, save=False,
             generation=None):
        size = self.imageSize
        img = Image.new('RGB', size, background)
        draw = Image.new('RGBA', size)
        pdraw = ImageDraw.Draw(draw)
        for shape in self.shapes:
            colour = shape.colour
            points = shape.points
            pdraw.polygon(points, fill=colour, outline=colour)
            img.paste(draw, mask=draw)

        if show:
            img.show()

        if save:
            fileName = u"{}".format(generation)
            saveLoc = u"{}.png".format(fileName)
            img = img.filter(ImageFilter.GaussianBlur(radius=3))
            img.save(saveLoc)

        return img

    def crossOver(self):
        shapes = copy.deepcopy(self.shapes)
        rand = random.randrange(0, len(shapes))
        newShape = shapes[rand]
        newShape.mutate(self.imageSize)
        return Chromosome(self.imageSize, shapes)

def fitness(img1, img2):
    img1List = np.frombuffer(img1.tobytes(), dtype=np.uint8)
    img1List = img1List.reshape((img1.size[1], img1.size[0], 3)) /255
    img2List = np.frombuffer(img2.tobytes(), dtype=np.uint8)
    img2List = img2List.reshape((img2.size[1], img2.size[0], 3)) /255
    difference = np.absolute(img1List - img2List) 
    difference = np.sum(difference,axis=(0,1))
    fitness = math.sqrt((difference[0]*difference[0])+(difference[1]*difference[1])+(difference[2]*difference[2]))
    return fitness

def rand_point(width, height):
    x = random.randrange(0 - 10, width + 10, 1)
    y = random.randrange(0 - 10, height + 10, 1)
    return (x, y)

def rand_color():
    red = random.randrange(0, 256)
    green = random.randrange(0, 256)
    blue = random.randrange(0, 256)
    alpha = random.randrange(0, 256)
    return (red, green, blue, alpha)

def initiateChromosome(imageSize, chromosomeSize=50, fixed_colour=False):
    chromosome = None
    shapes = []
    (width, height) = imageSize

    for i in range(POLYGONS):
        pointNo = random.randrange(POLY_MIN_POINTS, POLY_MAX_POINTS + 1)
        points = []
        for j in range(pointNo):
            point = rand_point(width, height)
            points.append(point)
        colour = COLOUR_WHITE if fixed_colour else rand_color()
        shape = Shape(colour, points)
        shapes.append(shape)
    chromosome = Chromosome(imageSize, shapes)
    return chromosome


In [ ]:
path = "mona1.jpeg"
img = Image.open(path)
imageSize = img.size
chromosome = initiateChromosome(imageSize, chromosomeSize=POLYGONS, fixed_colour=True)
parent = chromosome.draw(show=False)
parentFitness = fitness(img, parent)

generations = imageNo = 0
while True:
    mutatedChromosome = chromosome.crossOver()
    child = mutatedChromosome.draw()
    fitness_child = fitness(img, child)
    if fitness_child < parentFitness:
        chromosome = mutatedChromosome
        parentFitness = fitness_child
        # print("picking child w. fitness: {}".format(fitness_child))

    generations += 1
    if generations % 100 == 0:
        print("showing generation {}".format(generations))
        imageNo += 1
        chromosome.draw(show=True, save=True, generation=imageNo)

showing generation 100
showing generation 200
showing generation 300
showing generation 400
showing generation 500
showing generation 600
showing generation 700
showing generation 800
showing generation 900
showing generation 1000
showing generation 1100
showing generation 1200
showing generation 1300
showing generation 1400
showing generation 1500
showing generation 1600
showing generation 1700
showing generation 1800
showing generation 1900
showing generation 2000
showing generation 2100
showing generation 2200
showing generation 2300
showing generation 2400
showing generation 2500
showing generation 2600
showing generation 2700
showing generation 2800
showing generation 2900
showing generation 3000
showing generation 3100
showing generation 3200
showing generation 3300
showing generation 3400
showing generation 3500
showing generation 3600
showing generation 3700
showing generation 3800
showing generation 3900
showing generation 4000
showing generation 4100
showing generation 4200
s